In [1]:
import numpy as np
import pandas as pd

# 3 数据规整化：合并、清理、过滤

## 3.1 入门
#### 3.1.1 函数说明

In [2]:
df1 = pd.DataFrame([[1,2],[3,4]], index=['a', 'b'], columns=['A', 'B'])
df2 = pd.DataFrame([[5,6],[4,8]], index=['b', 'd'], columns=['B', 'C'])
df1

,A,B
a,1,2
b,3,4


In [3]:
df2

,B,C
b,5,6
d,4,8


如果单纯的按照index对齐，不如用concat方法，所以一般不推荐使用left_index, right_index。区别是concat不会对重名的列重命名。

In [4]:
pd.merge(left=df1, right=df2, how="inner", left_index=True, right_index=True)

,A,B_x,B_y,C
b,3,4,5,6


In [5]:
pd.merge(left=df1, right=df2, how="outer", left_index=True, right_index=True)

,A,B_x,B_y,C
a,1.0,2.0,NaN,NaN
b,3.0,4.0,5.0,6.0
d,NaN,NaN,4.0,8.0


In [11]:
d = pd.concat([df1, df2], axis=0).reset_index(drop=True)
d

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C
0,1.0,2,NaN
1,3.0,4,NaN
2,NaN,5,6.0
3,NaN,4,8.0


In [13]:
d.isnull().sum()

A    2
B    0
C    2
dtype: int64

In [6]:
pd.concat([df1, df2], join='inner', axis=1)

,A,B,B,C
b,3,4,5,6


#### 3.1.2 on用法

In [14]:
# 对于B列：df1中的'b'行，df2中的'd'行是相同的，其他都不同。相当于找到B列中元素相等的行，合并，并去调重复的列。
pd.merge(left=df1, right=df2, how='inner', on=['B'])

,A,B,C
0,3,4,8


In [16]:
# df1的A列a行，df2的C列b行是相同的，其他都不同。
# 其他列同名会进行重命名。
df1 = pd.DataFrame([[1,2],[3,4]], index=['a', 'b'], columns=['A', 'B'])
df2 = pd.DataFrame([[5,1],[4,8]], index=['b', 'd'], columns=['B', 'C'])
pd.merge(left=df1, right=df2,how='inner',left_on=['A'], right_on=['C'])

,A,B_x,B_y,C
0,1,2,5,1


#### 3.1.3 how 用法

In [17]:
# 保持左侧的DataFrame不变， 用右侧来跟它对齐，对不上的填NaN.
pd.merge(left=df1, right=df2, how='left', on=['B'])

,A,B,C
0,1,2,NaN
1,3,4,8.0


In [18]:
pd.merge(left=df1, right=df2, how='right', on=['B'])

,A,B,C
0,3.0,4,8
1,NaN,5,1


    对齐的列存在重复值
    重复的也没关系，操作逻辑是一致的，完全可以假想的 不存在的重复。

In [19]:
df1.loc['a', 'B'] = 4
df1

,A,B
a,1,4
b,3,4


In [20]:
pd.merge(left=df1, right=df2, how='right', on=['B'])

,A,B,C
0,1.0,4,8
1,3.0,4,8
2,NaN,5,1


## 3.2 多对一的合并
    一个表的连接键列有重复值， 另一个表中的连接键没有重复值。

In [21]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'], 'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [25]:
pd.merge(df1, df2)   # 默认为交

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [26]:
# 上面与这个等价： 内连接， 取交集
df1.merge(df2, on='key', how='inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [27]:
df1.merge(df2, on='key', how='outer')  # 外连接，取并集

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [28]:
df1.merge(df2, on='key', how='left')   # 左连接，左侧DataFrame取全部，右侧DataFrame取部分。

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [29]:
df1.merge(df2, on='key', how='right') 

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,a,2.0,0
4,a,4.0,0
5,a,5.0,0
6,d,NaN,2


 如果左右侧DataFrame的连接键列名不一致，但是取消有重叠，可使用left_on/right_on来指定左右连接键。

In [32]:
df3 = pd.DataFrame({'lkey': ['b','b','a','c','a','a','b'], 'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [33]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [34]:
df3.merge(df4, left_on='lkey', right_on='rkey', how='inner')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


## 3.3多对多的合并
#### 3.3.1 一个表的连接键列有重复值，另一个变种的连接键有重复值

In [35]:
df1 = df3 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'], 'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'b'], 'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,b,4


In [37]:
df1.merge(df2)

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,0,4
3,b,1,1
4,b,1,3
5,b,1,4
6,b,6,1
7,b,6,3
8,b,6,4
9,a,2,0


    df1 3a df5 2a + df1 3b df5 3b ==> 15
    合并小结
    1）默认情况下，会将两个表中相同列名作为连接键
    2）多对多，会采用笛卡尔积形式链接（左表连接键有三个值‘1，3，5’，右表有两个值‘2，3’，则会形成(1,2),(1,3),(3,1),(3,2)...6种组合）在如，假设集合A={a, b}，集合B={0, 1, 2}，则两个集合的笛卡尔积为{(a, 0), (a, 1), (a, 2), (b, 0), (b, 1), (b, 2)}。